<a href="https://colab.research.google.com/github/MaxManfred/deep-learning/blob/master/notebooks/Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Two-class classification, or binary classification**, may be the most widely applied kind of machine-learning problem. In this example, you’ll learn to classify movie reviews as positive or negative, based on the text content of the reviews.

You’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. 
They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.
Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.

In [0]:
# import needed libraries
from keras.datasets import imdb
import numpy as np

Load IMDB data set in Keras.

The argument num_words=10000 means you’ll only keep the top 10,000 most  frequently occurring words in the training data. Rare words will be discarded. 
This allows you to work with vector data of manageable size.
The variables train_data and test_data are lists of reviews; each review is a list of word indices (encoding a sequence of words). 
train_labels and test_labels are lists of 0s and 1s, where 0 stands for negative and 1 stands for positive: bold text

In [0]:
(training_samples, training_labels), (test_samples, test_labels) = imdb.load_data(num_words=10000)

We print the first sample review in the training set and its label.
We extract word_index from IMDB: word_index is a dictionary mapping words to an integer index.
We then create reverse_word_index which reverses word_index, mapping integer indices to words.
Note that the indices are offset by 3 because 0, 1, and 2 are reserved indices for “padding,” “start of sequence,” and “unknown.”.
Finally we print the decoded review.

In [41]:
# Because you’re restricting yourself to the top 10,000 most frequent words, no 
# word index will exceed 10,000
print('Highest index of selected terms in the reviews: ', 
      max( [max(review) for review in training_samples] ))

# as an example, show a sample
print('Training sample: ', training_samples[0])
print('Training label: ', training_labels[0])

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') + '(' + str(i - 3) + ')' for i in training_samples[0]])
print('Decoded training sample: ', decoded_review)

Highest index of selected terms in the reviews:  9999
Training sample:  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 1

You can’t feed lists of integers into a neural network. 
You have to turn your lists into tensors. 
One-hot encode your lists to turn them into vectors of 0s and 1s. 
This would mean, for instance, turning the sequence [3, 5] into a 10,000-dimensional vector that would be all 0s except for indices 3 and 5, which would be 1s. 
Then you could use as the first layer in your network a Dense layer, capable of handling floating-point vector data.

In [0]:
# define the vectorization process as a function
def vectorize_reviews(reviews, dimension = 10000):
    results = np.zeros((len(reviews), dimension))
    for i, review in enumerate(reviews):
        results[i, review] = 1.
    return results
  
# vectorize samples
vectorized_training_samples = vectorize_reviews(training_samples)
vectorized_test_samples = vectorize_reviews(test_samples)

# vectorize labels
vectorized_training_labels = np.asarray(training_labels).astype('float32')
vectorized_test_labels = np.asarray(test_labels).astype('float32')

In [60]:
# print a vectorized sample
# print(vectorized_training_samples[0])

# create a set from the first review and sort it in ascending order to print the 
# indices of the terms to compare them with the vectorization
first_sample = list(set(list.copy(training_samples[0])))
first_sample.sort()
print('First 100 sorted term indices in first training sample: ', first_sample[0:100])
print('First 100 coordinated of the vectorization of the first training sample: ', vectorized_training_samples[0][0:100,])



First 100 sorted term indices in first training sample:  [1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 25, 26, 28, 30, 32, 33, 35, 36, 38, 39, 43, 46, 48, 50, 51, 52, 56, 62, 65, 66, 71, 76, 77, 82, 87, 88, 92, 98, 100, 103, 104, 106, 107, 112, 113, 117, 124, 130, 134, 135, 141, 144, 147, 150, 167, 172, 173, 178, 192, 194, 215, 224, 226, 256, 283, 284, 297, 316, 317, 336, 381, 385, 386, 400, 407, 447, 458, 469, 476, 480, 515, 530, 546, 619, 626, 670, 723, 838, 973, 1029, 1111, 1247]
First 100 coordinated of the vectorization of the first training sample:  [0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0.]


The input data is vectors, and the labels are scalars (1s and 0s): this is the easiest setup you’ll ever encounter. 
A type of network that performs well on such a problem is a simple stack of fully connected (Dense) layers with relu activations: Dense(16, activation='relu').

The argument being passed to each Dense layer (16) is the number of hidden units of the layer. 
A hidden unit is a dimension in the representation space of the layer.
Each such Dense layer with a relu activation implements the following chain of tensor operations:

`output = relu(dot(W, input) + b)`

Having 16 hidden units means the weight matrix W will have shape (input_dimension, 16): the dot product with W will project the input data onto a 16-dimensional representation space (and then you’ll add the bias vector b and apply the relu operation). You can intuitively understand the dimensionality of your representation space as “how much freedom you’re allowing the network to have when learning internal representations.” 
Having more hidden units (a higher-dimensional representation space) allows your network to learn more-complex representations, but it makes the network more computationally expensive and may lead to learning unwanted patterns (patterns that will improve performance on the training data but not on the test data).

There are two key architecture decisions to be made about such a stack of Dense layers:

*   How many layers to use
*   How many hidden units to choose for each layer

Later on you’ll learn formal principles to guide you in making these choices. 
For the time being, let's use the following architecture choice:

*   Two intermediate layers with 16 hidden units each
*   A third layer that will output the scalar prediction regarding the sentiment of the current review

The intermediate layers will use **relu** as their activation function, and the final layer will use a **sigmoid** activation so as to output a probability (a score between 0 and 1, indicating how likely the sample is to have the target “1”: how likely the review is to be positive).

![alt text](https://drive.google.com/open?id=1ecyoCs1eUZoBkmVf3a1rNjIJH0lPfNQB)